# Phase II Model Test
First attempt at a Phase II model, one which creases the master schedule and assigns students to courses simultaneously.

$S$ -- Set of all students

$C$ -- Set of all courses

$T$ -- Set of all periods {1,2,3,4,7,8}

$I$ -- Set of all instructors

**Variables:**

$x_{i,j,t}$ for  $i \in S, j \in C, t \in T$ -- Binary, 1 if student $i$ assigned to course $j$ in period $t$

$c_{j,t}$ for $j \in C, t \in T$ -- Binary, 1 if course $j$ to be offered in period $t$

$t_{k,j,t}$ for $k \in I, j \in C, t \in T$ -- Binary, 1 if teacher $k$ teaching course $j$ in period $t$

** Parameters:**

$P_{i,j}$ -- Preference for student $i$ on course $j$

$S_i$ -- Seniority constant, e.g., higher for seniors

$D_{i,j}$ -- Binary, 1 if course $i$ and $j$ are in the same department, i.e., if they meet the same requirement, e.g., highschool math

$Ta_{i,j}$ -- Binary, 1 if teacher $i$ is teaching course $j$

$Cap_j$ -- Capacity of course $j$


** Constraints: **

$\sum_{j} x_{i,j,t} =1 \quad \forall i \in S, t \in T$ -- Says students can be assigned to exactly one course per period.

$\sum_{t \neq T} \sum_{i \in C} \sum_{j \neq i \in C} D_{i,j} x_{k,i,t} = 0 \quad \forall k \in S, T \in T$ -- Says that students are not assigned to more than 1 class in each department (could always change to more nuanced number of courses per department--how???)

$\sum_{t \in T} c_{j,t} = 1 \quad \forall j \in C$ -- Says that each course taught only once

$\sum_{i \in S} x_{i,j,t} \leq Cap_j \quad \forall j \in C, t \in T$ -- Course capacity constraint

$\sum_{j \in C} c_{j,t} Ta_{k,j} \leq 1  \quad \forall k \in I, \forall t \in T$ -- Teacher constraint (a teacher can teach at most one course per period), where $Ta_{k,j}$ is a parameter, not a variable

** Objective: **

$ \text{max }\sum_{i \in S} \sum_{j \in C} x_{i,j} P_{i,j} $ -- Assuming preferences take a higher value if they are a student's preferred choice, this will give a higher weight to higher assignments (at this point, I am leaving out the seniority multiplier).

In [15]:
from pyscipopt import Model, quicksum
import numpy as np

In [16]:
# set up some fake data
## Sets
S = [0,1,2,3,4,5] # 6 students
C = [0,1,2,3] # 4 courses
T = [0,1] # 2 periods
I = [0,1] # 2 instructors

## Preferences
#     0    1    2    3  -- Courses
P = [[1,   0,   1,   0], # student 0
     [0,   1,   1,   0], # student 1
     [0,   1,   0,   1], # student 2
     [1,   0,   0,   1], # student 3
     [1,   0,   1,   0], # student 4
     [0,   1,   1,   0]] # student 5

## Capacity
Cap = [3, 3, 3, 3]

## Course Proximity
#     0    1    2    3  -- Courses
D = [[0,   0,   0,   0], # 0 
     [0,   0,   1,   0], # 1
     [0,   1,   0,   1], # 2
     [1,   0,   0,   0]]

## Teacher Assignments
#     0    1    2    3  -- Courses
Ta = [[1,   0,   1,  0],  # Teacher 1
     [0,   1,   0,  1]]  # Teacher 2

In [17]:
# Setup model
m = Model("test")

## Note, in the below I do not have a period index on students, I couldn't figure out how to work it

In [18]:
# Add Student Variables
X = {} # Variable dictionary
for i in S:
    for j in C:
            name = "Student " + str(i) + ", in course " + str(j)
            X[i,j] = m.addVar(name, vtype = 'B')

In [19]:
# Add Course Variable
Course = {} # Variable dictionary
for j in C:
    for t in T:
        name = "Course " + str(j) + " in period " + str(t)
        Course[j,t] = m.addVar(name, vtype='B')

In [20]:
# Add Student assignment constraint
for i in S:
        m.addCons(quicksum(X[i,j] for j in C) == 1)

In [21]:
# Add capacity constraint
for j in C:
    m.addCons(quicksum(X[i,j] for i in S) <= Cap[j])

In [22]:
# Add course proximity constraint (without quicksum)
for k in S:
    expr = 0 # reset expression (should not have impact)
    for i in C:
        small_set = list(set(C) - set([int(i)])) # C - {i} list of courses without course i
        for j in small_set:
            expr = expr + D[i][j]*X[k,i]  
    m.addCons(expr == 0)

In [24]:
# Teacher Constraint
for k in I:
    for t in T:
        m.addCons(quicksum(Course[j,t]*Ta[k][j] for j in C) <= 1)

In [25]:
# Course Taught only once Constraint
for j in C:
    m.addCons(quicksum(Course[j,t] for t in T) == 1)

In [30]:
# Set objective
m.setObjective(quicksum(X[i,j]*P[i][j] for i in S for j in C), "maximize")

In [31]:
# Solve model
m.optimize()

In [45]:
# Look at output
for i in S:
    for j in C:
        s = str(X[i,j])
        if m.getVal(X[i,j]) == 1:
            print(s + "\tyes")
        else:
            print(s + "\tno")
            
print("\n\n")
for j in C:
    for t in T:
        s = str(Course[j,t])
        if m.getVal(Course[j,t]) == 1:
            print("Course " + str(j) + " to be taught in period ", str(t))

Student 0, in course 0	yes
Student 0, in course 1	no
Student 0, in course 2	no
Student 0, in course 3	no
Student 1, in course 0	yes
Student 1, in course 1	no
Student 1, in course 2	no
Student 1, in course 3	no
Student 2, in course 0	yes
Student 2, in course 1	no
Student 2, in course 2	no
Student 2, in course 3	no
Student 3, in course 0	yes
Student 3, in course 1	no
Student 3, in course 2	no
Student 3, in course 3	no
Student 4, in course 0	yes
Student 4, in course 1	no
Student 4, in course 2	no
Student 4, in course 3	no
Student 5, in course 0	yes
Student 5, in course 1	no
Student 5, in course 2	no
Student 5, in course 3	no



Course 0 to be taught in period  0
Course 1 to be taught in period  0
Course 2 to be taught in period  0
Course 3 to be taught in period  0
